# Word Sense Disambiguation using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 15 different words. 

In [2]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext

# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:3')

batch_size = 128
learning_rate = 0.001
epochs = 3

# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 15 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [3]:
from random import shuffle
from math import floor

def data_split(path_to_dataset, split = 0.8):
    '''Splits txt into random 80:20 subsets and saves them as csv'''
    
    #Shuffle and split
    with open(path_to_dataset, encoding='utf8') as f:
        lines = [l for l in f]
        shuffle(lines)
        split_index = floor(len(lines) * split)
        training,testing = lines[:split_index],lines[split_index:]
     
    #Save training csv
    with open(path_to_dataset.replace('.txt', '-train.csv'), 'w', encoding='utf8') as f:
        for l in training:
            f.write(l)
            
    #Save testing csv
    with open(path_to_dataset.replace('.txt', '-test.csv'), 'w', encoding='utf8') as f:
        for l in testing:
            f.write(l)

### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [4]:
def mcs_baseline(data):
    words = {}
    with open(data, encoding='utf8') as f:
        for l in f:
            sense, form, idx, context = l.split('\t')
            if form not in words:
                words[form] = {}
            words[form][sense] = words[form].get(sense,0)+1
    
    words_baseline = {}
    for form, senses in words.items():
        mostcommon, freq = sorted(senses.items(), key= lambda item:item[1], reverse=True)[0]
        total_freqs = sum(senses.values())
        baseline = freq/total_freqs  #Occurrence of the most common sense / occurrances of all senses
        wdict = words_baseline[form[:form.index('.')] ] = {}
        wdict['mcs'] = baseline
        wdict['senses'] = len(senses)
    
    return words_baseline

mcs_baseline('wsd-data/wsd_data.txt') # key:value = lemma:MCS (ie, Most common sense count / All senses counts )

{'keep': {'mcs': 0.3919896161691081, 'senses': 11},
 'national': {'mcs': 0.20456663560111835, 'senses': 6},
 'build': {'mcs': 0.21202404809619238, 'senses': 10},
 'place': {'mcs': 0.2428793371310202, 'senses': 7},
 'position': {'mcs': 0.20171094101755965, 'senses': 6},
 'serve': {'mcs': 0.1551155115511551, 'senses': 9},
 'hold': {'mcs': 0.15195137555982086, 'senses': 11},
 'line': {'mcs': 0.8512173128944995, 'senses': 11},
 'see': {'mcs': 0.6275619455490976, 'senses': 11},
 'time': {'mcs': 0.27864214992927866, 'senses': 5},
 'physical': {'mcs': 0.23641160949868073, 'senses': 6},
 'follow': {'mcs': 0.14589020321504398, 'senses': 11},
 'regular': {'mcs': 0.21732522796352582, 'senses': 8},
 'bad': {'mcs': 0.6073903002309469, 'senses': 4},
 'force': {'mcs': 0.16273156556483837, 'senses': 8},
 'professional': {'mcs': 0.21756487025948104, 'senses': 5},
 'security': {'mcs': 0.2033271719038817, 'senses': 7},
 'positive': {'mcs': 0.35444078947368424, 'senses': 5},
 'point': {'mcs': 0.3554626241

### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()* 

In [5]:
from torchtext.data import Field, BucketIterator, TabularDataset
def dataloader(path, batch_size=8):
    #Fields: word-sense, lemma, position,  context
    Tokens = torchtext.data.Field(tokenize=lambda x:x.split(), lower=True, batch_first=True) #TODO lowercase?
    Labels = torchtext.data.Field(batch_first=True) #TODO trim ".pos" or not?
    
    fields = [('sense',Labels),('lemma',Tokens),('position',Tokens),('context',Tokens)]
    
    #Process from csv files
    train,test = torchtext.data.TabularDataset.splits(
            path=path, train='wsd_data-train.csv', test='wsd_data-test.csv',
            format='csv', fields=fields, skip_header=False, 
            csv_reader_params = {'delimiter':'\t','quotechar':'、'})
    
    #Build vocab
    Labels.build_vocab(train) # nr of classes
    Tokens.build_vocab(train)

    #Batch iterator
    train_iter,test_iter = torchtext.data.BucketIterator.splits(
            (train,test), batch_size=batch_size, shuffle=True, device=device,
             sort_within_batch=True, sort_key=lambda x: len(x.context))
    
    return train_iter, test_iter, Tokens.vocab, Labels.vocab
        #  train_iter, test_iter, vocab, labels  

# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


In [6]:
# TODO model:  given lemma/word at position in context => which word sense 
#                                                      (out of 224 senses or out of the set of given word senses?)

In [7]:
#  Given the index of target word (column 3 in the dataset) => predict the word sense
#  QUE: the index number or the 'indexed token'?

def get_target_token(bat):
    '''Given the index "n" from column 3, select the nth token in the context sequence.'''
    ctx, position = bat.context, bat.position
    targets=[]
    for n in range(len(bat)):
        seq = ctx[n] #the nth context seq in each batch
        target_i = vocab.itos[position[n]] 
        target_i = int(target_i) # the index integer for the target token in the nth seq
        target_token = seq[target_i]
        targets.append([target_token])

    return torch.tensor(targets).to(device) # B,1 where dim1 is the target token index

class WSDModel_approach1(nn.Module):
    def __init__(self, vocab_dim, num_senses, h_dim):
        super(WSDModel_approach1, self).__init__()
        self.embeddings = nn.Embedding(vocab_dim, h_dim)
        self.rnn = nn.LSTM(h_dim, h_dim, batch_first=True)
        self.classifier = nn.Linear(h_dim, num_senses) # TODO total senses or a word's senses?
#         self.dropout = nn.Dropout(0.2)
    
    def forward(self, batch):
        
        # Select column 3 => Index the ith token in the context        
        batch_targets = get_target_token(batch) # B,1
        
        #Embed the input
        embedded = self.embeddings(batch_targets) # B,1,HD
        
        #Put the embeddings through RNN
        rep, (_, _) = self.rnn(embedded)
#         rep = self.dropout(rep)
        
        # Predict the class from RNN output
        predictions = self.classifier(rep)
        
        return predictions

In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [8]:
# Given "final hidden state of RNN" => predict word sense
class WSDModel_approach2(nn.Module):
    def __init__(self, vocab_dim, num_senses, h_dim):
        # your code goes here
        super(WSDModel_approach2, self).__init__()
        self.embeddings = nn.Embedding(vocab_dim, h_dim)
        self.rnn = nn.LSTM(h_dim, h_dim, batch_first=True)
        self.classifier = nn.Linear(h_dim, num_senses) # TODO total senses or a word's senses?
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, batch):
        # Select columns
        position, lemma, context = batch.position, batch.lemma, batch.context
        
        #Embed the inputs => # B,SeqLen,HD
        position, lemma, context = self.embeddings(position), self.embeddings(lemma), self.embeddings(context)
        
        #Merge the embeddding tensors by concat & multiplying
        multiplication = position*lemma*context # B,S,HD
        fullcontext = torch.cat([position, lemma, context, multiplication], dim=1) # B,S*4,HD

        #Put the embeddings through RNN
        rep, (_, _) = self.rnn(fullcontext)
        rep = torch.sum(rep, dim=1) # B,CtxLen,HD => B,HD
        rep = self.dropout(rep)
        
        # Predict the class from RNN output
        predictions = self.classifier(rep)
        
        return predictions

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [16]:
# Split raw data => Load train & test sets
data_split('wsd-data/wsd_data.txt')
train_iter, test_iter, vocab, labels = dataloader( 'wsd-data/', batch_size=batch_size )

In [ ]:
#Params
vocab_dim,num_labels = len(vocab),len(labels)  # V=70521, L=224
h_dim = 256  # Or use in_dim=32 during dev...?

wsd1_model = WSDModel_approach1(vocab_dim, num_labels, h_dim)
wsd1_model.to(device)
wsd2_model = WSDModel_approach2(vocab_dim, num_labels, h_dim)
wsd2_model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(wsd1_model.parameters(), lr=learning_rate)
optimizer2 = torch.optim.Adam(wsd2_model.parameters(), lr=learning_rate)

#Training 1st model
from statistics import mean
total_loss1, total_loss2 = [],[]
for _ in range(epochs):
    
    total_loss1.clear()
    total_loss2.clear()
    
    for i, batch in enumerate(train_iter):
        
        label = batch.sense  # gold labels of batch
        output1 = wsd1_model(batch)
        output2 = wsd2_model(batch)
        
#         print(output.view(-1, num_labels).shape) # 8,1,224
#         print(label.shape)
#         break
        
        loss1 = loss_function(output1.view(-1, num_labels), label.view(-1)) # modelout:(B,num_labels), target:(B,1)
        total_loss1 += [loss1.item()]
        
        loss2 = loss_function(output2.view(-1, num_labels), label.view(-1)) # modelout:(B,num_labels), target:(B,1)
        total_loss2 += [loss2.item()]
        
        print(f'Avg total loss 1: {mean(total_loss1)}  Avg total loss 2: {mean(total_loss2)}', end='\r')
        
        # compute gradients; # update parameters; # reset gradients
        loss1.backward();     optimizer1.step();    optimizer1.zero_grad()
        loss2.backward();     optimizer2.step();    optimizer2.zero_grad()
    
    print()

In [23]:
# test model after all epochs are completed
# Test WSD1&2 with the test_iter
test_loss1,test_loss2 = [],[]
wsd1_model.eval()
wsd2_model.eval()

# iterate over the test data and compute the class probabilities, same
# procedure as before, but now we don't backpropagate
for i, batch in enumerate(test_iter):
    
    label = batch.sense  # gold labels of batch

    with torch.no_grad(): # dont collect gradients when testing
        # Given batch => Predict label
        output1, output2 = wsd1_model(batch), wsd2_model(batch)
    batch_loss1 = loss_function(output1.view(-1,num_labels), label.view(-1))
    batch_loss2 = loss_function(output2.view(-1,num_labels), label.view(-1))
    test_loss1 += [batch_loss1.item()]
    test_loss2 += [batch_loss2.item()]

    print('Avg test loss 1: ', mean(test_loss1), '  Avg test loss 2: ', mean(test_loss2), end='\r')

In [20]:
# Save:
# torch.save(wsd1_model.state_dict(), 'wsd1_model.pt')
# torch.save(wsd2_model.state_dict(), 'wsd2_model.pt')

# Load:
_, test_iter, vocab, labels = dataloader( 'wsd-data/', batch_size=128 )
del _
vocab_dim,num_labels = len(vocab),len(labels)  # V=70521, L=224
h_dim = 256

wsd1_model = WSDModel_approach1(vocab_dim, num_labels, h_dim)
wsd1_model.load_state_dict(torch.load('wsd1_model.pt'), )
wsd1_model.eval()

wsd2_model = WSDModel_approach2(vocab_dim, num_labels, h_dim)
wsd2_model.load_state_dict(torch.load('wsd2_model.pt'), )
wsd2_model.eval()

WSDModel_approach2(
  (embeddings): Embedding(70219, 256)
  (rnn): LSTM(256, 256, batch_first=True)
  (classifier): Linear(in_features=256, out_features=224, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the BertTokenizer to transform the sentence into integers. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either the the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [10]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [36]:
#Turn each batch (batchsize of lines) into an obj, where each column is an attribute

def bert_tensor(seq, max_length=16):
    tensor = tokenizer.encode(seq, return_tensors='pt', max_length=max_length, pad_to_max_length=True)
    return tensor

def bert_attn_mask(seq, max_length=16):
    tensor = tokenizer.encode_plus(seq, return_tensors='pt', max_length=max_length, pad_to_max_length=True,
                                   return_attention_mask=True)
    return tensor['attention_mask']

def bert_tensor_no_pad(seq):
    tensor = tokenizer.encode(seq, return_tensors='pt')
    return tensor
    
class Batch():
    def __init__(self, batchlines, dictionary):
        self.sense = torch.tensor( [ [dictionary[l[0]]] for l in batchlines] ).to(device)
        self.context = torch.stack( [bert_tensor(l[3], 384) for l in batchlines], dim=1 )[0].to(device)
        self.context_attn_mask = torch.stack( [bert_attn_mask(l[3], 384) for l in batchlines], dim=1 )[0].to(device)
        
from random import shuffle
def dataloader_for_bert(path_to_file, batch_size):
    '''Shuffles lines of file, then yield a batch'''
    sensevocab = [] 
    
    # Iterate through the rows
    with open(path_to_file, encoding='utf8') as f:
        lines = []  # 76049 lines 
        for l in f:
            columns = sense, lemma, position, context = l.rstrip('\n').split('\t')
            
            if sense not in sensevocab:
                sensevocab.append(sense)
            
            lines.append(columns)
    
    #Convert list of senses to dictionary of indexed senses
    sensevocab_itos = dict( enumerate(sensevocab) )
    sensevocab_stoi = {x:i for i,x in sensevocab_itos.items()}
    
    #Shuffle rows and make generator
    shuffle(lines)
    n = max(1, batch_size)
    generator = (Batch(lines[i:i+n], sensevocab_stoi) for i in range(0, len(lines), n))
        
    return generator, sensevocab_itos

In [37]:
generator, sensevocab = dataloader_for_bert('wsd-data/wsd_data-train.csv',8)

In [50]:
for i, bat in enumerate(generator):
#     ctx = bat.context
#     ctx_attn_mask = bat.context_attn_mask
# #     print( tokenizer.convert_ids_to_tokens(ctx[0]) ) # tensor=>list of strings
#     print( tokenizer.decode(ctx[0]) ) # tensor=> sequence string
#     print(ctx_attn_mask[0]) # [1,1,1...0,0,0] where 1 means other tokens and 0 means [PAD]
    print(bat.sense)
    print( [ sensevocab[int(i)] for i in bat.sense.view(-1)] )
    break

tensor([[156],
        [ 99],
        [135],
        [ 84],
        [ 96],
        [102],
        [ 17],
        [161]])
['position%1:15:00::', 'active%5:00:00:operational:00', 'force%1:07:01::', 'lead%2:38:00::', 'life%1:26:01::', 'case%1:26:00::', 'case%1:11:00::', 'national%3:00:00::']


In [12]:
from transformers import BertModel
class BERT_WSD(nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super(BERT_WSD, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier =  nn.Sequential( nn.Linear(in_dim, h_dim), nn.ReLU(), nn.Linear(h_dim, out_dim) )
        
    def forward(self, bat):
        ctx, ctx_attn, sense = bat.context, bat.context_attn_mask, bat.sense
        input_ids, attnmask, label = ctx.to(device), ctx_attn.to(device), sense.to(device)
        outputs = self.bert(input_ids=input_ids, attention_mask=attnmask) #([B,Seq,768], [B,768])
        first_hidden_state_cls = outputs[0][:, 0, :] # B, 768 => [CLS] token of last layer
        predictions = self.classifier(first_hidden_state_cls)  # B, 222
        return predictions

In [13]:
batch_size = 16
device=torch.device('cuda:3')

# _, sensevocab = dataloader_for_bert('wsd-data/wsd_data-train.csv', batch_size)

In [14]:
bert_lr = 2e-5 # -4~-6

from transformers import AdamW

bert_wsd_model = BERT_WSD(768,50, len(sensevocab))
loss_function = nn.CrossEntropyLoss()
optimizer = AdamW(bert_wsd_model.parameters(), lr=bert_lr, eps = 1e-8)
bert_wsd_model.to(device)


# train model
from statistics import mean
train_loss = []
for _ in range(epochs):
    
    #The generator is exhausted at the end of an epoch so needs to be remade
    train_generator, _ = dataloader_for_bert('wsd-data/wsd_data-train.csv', batch_size)

    train_loss.clear()
    bert_wsd_model.train()
    
    for i,bat in enumerate(train_generator):
        
        label = bat.sense.to(device)
        output = bert_wsd_model(bat)
#         print(label.shape) #B, 1
#         print(output.shape) # B,222
#         break
        
        loss = loss_function( output, label.view(-1) ) 
        train_loss.append( loss.item() )

        print(f'Average total loss: {mean(train_loss)}', end='\r')
        
        torch.nn.utils.clip_grad_norm_(bert_wsd_model.parameters(), 1.0)
        
        # compute gradients; # update parameters; # reset gradients
        loss.backward();     optimizer.step();    optimizer.zero_grad()
    
    print()

Average total loss: 2.7584107885259157
Average total loss: 1.1459777694163182
Average total loss: 0.7890831329478987


In [24]:
# Save:
# torch.save(bert_wsd_model.state_dict(), 'bert_model.pt')

# Load:
_, sensevocab = dataloader_for_bert('wsd-data/wsd_data-train.csv', 128)
bert_wsd_model = BERT_WSD(768,50, len(sensevocab))
bert_wsd_model.load_state_dict(torch.load('bert_model.pt'), )
bert_wsd_model.eval()

BERT_WSD(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

# 3. Evaluation

In [70]:
# EVALUATION OF MODELS (compare to each other and the MCS baselines)

import numpy as np
import pandas as pd

def eval_models():
    # Run both models on test data and count the correct vs total predictions of each lemma
    dict1, dict2, dict3 = {},{},{}
    for index,bat in enumerate(test_iter):
        label = bat.sense
        
        with torch.no_grad(): # dont collect gradients when testing
            # B,1,L
            output1, output2 = wsd1_model(bat), wsd2_model(bat)

        #Iterate through the predictions & gold labels of a batch
        for output,dictionary in [(output1,dict1), (output2,dict2)]:
            for i in range(len(bat)):
                idx_of_max_val = torch.argmax(output[i].view(-1))
                prediction = labels.itos[idx_of_max_val]

                gold = labels.itos[label[i]]
                goldlemma = gold[:gold.index('%')]

                if goldlemma not in dictionary:
                    dictionary[goldlemma]=[0,0] # correct_predictions, total_count of a word lemma
                w = dictionary[goldlemma]
                w[0]+=int(gold==prediction)
                w[1]+=1
                
    for index,bat in enumerate(test_generator):
        label = bat.sense
        
        with torch.no_grad(): # dont collect gradients when testing
            output = bert_wsd_model(bat)

        #Iterate through the predictions & gold labels of a batch
        for i in range(bat.sense.shape[0]):
            idx_of_max_val = torch.argmax(output[i])
            prediction = sensevocab[ int(idx_of_max_val) ]

            gold = sensevocab[ int(label[i]) ]
            goldlemma = gold[:gold.index('%')]

            if goldlemma not in dict3:
                dict3[goldlemma]=[0,0] # correct_predictions, total_count of a word lemma
            w = dict3[goldlemma]
            w[0]+=int(gold==prediction)
            w[1]+=1
        

    # Compute the lemmas' respective accuracies and compare with MCS baselines
    baselinedict = mcs_baseline('wsd-data/wsd_data.txt')  
    stats = {} # The per-word-form accuracy
    for lemma in baselinedict:
        accu = lambda duos : [duo[0]/duo[1] for duo in duos]
        accuracy1, accuracy2, accuracy3 = accu( [dict1[lemma], dict2[lemma], dict3[lemma]] )

        mcsdict = baselinedict[lemma]
        mcs = mcsdict['mcs']
        numsenses = mcsdict['senses']
        res = lambda accuracy : '🙂' if accuracy>mcs else '🤢'  # Marks whether it's better or worse than MCS

        stats[lemma] = {
            'LSTM 1 accuracy':f'{round(accuracy1,3)}{res(accuracy1)}',
            'LSTM 2 accuracy':f'{round(accuracy2,3)}{res(accuracy2)}',
            'BERT accuracy':f'{round(accuracy3,3)}{res(accuracy3)}',
            'MCS baseline':round(mcs,3), 
            'Nr of senses':numsenses
        }

    df = pd.DataFrame.from_dict(stats, orient='index').sort_values(by='Nr of senses', ascending=False)
    return df


wsd1_model.to(device); wsd2_model.to(device); bert_wsd_model.to(device)
_, test_iter, vocab, labels = dataloader( 'wsd-data/', batch_size=128 )
test_generator, sensevocab = dataloader_for_bert('wsd-data/wsd_data-test.csv', 128)

eval_models()

,LSTM 1 accuracy,LSTM 2 accuracy,BERT accuracy,MCS baseline,Nr of senses
keep,0.569🙂,0.715🙂,0.91🙂,0.392,11
hold,0.199🙂,0.561🙂,0.854🙂,0.152,11
see,0.612🤢,0.732🙂,0.915🙂,0.628,11
line,0.862🙂,0.934🙂,0.987🙂,0.851,11
follow,0.367🙂,0.559🙂,0.798🙂,0.146,11
build,0.23🙂,0.321🙂,0.611🙂,0.212,10
find,0.375🙂,0.52🙂,0.866🙂,0.232,10
serve,0.213🙂,0.549🙂,0.816🙂,0.155,9
life,0.213🤢,0.65🙂,0.915🙂,0.225,9
lead,0.212🙂,0.412🙂,0.781🙂,0.180,8


Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

**Ans:**

**LSTM 1:**
Originally we chose 'column 3' as instructed but it doesn't make sense to give the integer and predict the sense. So we used the integer to select the i-th token id in the context, which means the models only gets the word (could be its variant forms instead of the lemma) and tries to predict the sense. 

Eg, Given position='4' and context=`[please stay in the lines]`, the model gets the vocab id for `'lines'` and tries to predict a sense out of all the word senses, including out of those for `'line'`.

**LSTM 2:**
The second model is given the full context, ie, a combined tensor of `[position, lemma, context, position x lemma x context]`. The models performs better than the MCS for each lemma.

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

**Ans:**

Model 2 performs better than model 1 and the MCS. 
In the cases where model 1 performs worse than the MCS, the accuracies are mostly just below the MCS, suggesting that it outputs the most common sense of the word.

Regarding the number of word sense, model 1 has instances where it performs worse the MCS for both words with fewer and more senses, so we conclude that the number of word senses doesn't necessarily play a role in the accuracy.


How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

**Ans:**

Both the LSTM2 model and BERT performed well, but BERT has 80~90 percent accuracy for most words. Note that we only gave the context sentence as input, without combining the position or the lemma tensors (which could've been done and possibly helped with the performance) because of the efficiency considerations and the complexity of having to also deal with attention masks).

**Diffrences in representations**

When tokenizing the data for LSTM, we only separate and lowercase the words and build an indexed vocabulary, which would ineviably contain various forms of the same word.

On the other hand, the data for BERT uses the prebuild tokenizer and BERT vocab, where the words are broken down into subwords in the BERT vocab. Thus different words in the input can be represented as lemma+##suffix instead of different vocab entries as in the case of LSTM.

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

**Ans:**

Based on the inputs we gave to the LSTM models, the informations given to the model training and the way the representation are combined greatly affects the performance. Apart from formatting the input, we can also implement deeper layers in the model to extract further information from the more general data from previous layers. 

As for BERT fine tuning, it may be beneficial to utilize the pretrained model and its vocabulary. In our model, for example, we represented the word senses in a separate vocab rather using the BERT tokenizer, but it may be a good idea to let context and sense share the same BERT vocab, since it can break words down to smaller subwords, so that the model can learn a better relationship between the sense and the context. Eg, with 'keep'+'##s' in the context, it should predict a sense that starts with 'keep'+'%'. Another benefit of tokenizing to the BERT vocab is that the token ids will be consistent across different datasets (instead of built from our own data), which will facillitate evaluating and comparing a variety of datasets and models.

# Readings:

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf